In [1]:
import pandas as pd
import jsonlines

In [2]:
%load_ext autoreload
%autoreload 2

from seewhence.emoji import *

# Load Sarcasm Data

In [17]:
reddit_path = '/home/ben/data/acl_2020/reddit.jsonl'
twitter_path = '/home/ben/data/acl_2020/twitter.jsonl'

with jsonlines.open(twitter_path) as twitter_reader:
    _twitter = pd.DataFrame([x for x in twitter_reader])
    _twitter['source'] = 'twitter'

with jsonlines.open(reddit_path) as reddit_reader:
    _reddit = pd.DataFrame([x for x in reddit_reader])
    _reddit['source'] = 'reddit'

# add multiindex for hygiene
# df = pd.concat([_twitter,_reddit], keys=['twitter','reddit'])

df = pd.concat([_twitter,_reddit]).reset_index(drop=True)

In [18]:
# make classification easier with binary
df['label'] = df.label.apply(lambda x: 1 if x == 'SARCASM' else 0)
df['context_0'] = df.apply(lambda x: x.context[0], axis=1)
df['context_1'] = df.apply(lambda x: x.context[1], axis=1)
df['combined'] = df.apply(lambda x: '\n'.join(x.context + [x.response]), axis=1)

## Deal with Emojis

In [34]:
text_cols = ['response','context_0','context_1']

for col in text_cols:
  df[col+'_emoji_unicode'] = df[col].apply(lambda x : ' '.join(get_emojis(x)))
  df[col+'_emoji_string'] = df[col].apply(lambda x : ' '.join(clean_emojis(translate_emojis(x))))

emoji_text_cols = [col for col in df.columns if 'emoji_string' in col]
emoji_cols = [col for col in df.columns if 'emoji_unicode' in col]

NameError: name 'DO_EMOJI' is not defined

In [45]:
%load_ext autoreload
%autoreload 2

from seewhence import models
from seewhence import train

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Splits

In [19]:
train_split = 0.8 if not train_split else train_split

In [27]:
train_size = int(len(df) * train_split)
test_size = int(len(df) - train_size)//2

In [26]:
df = df.sample(frac=1)
twitter = df.loc[df.source == 'twitter']
reddit = df.loc[df.source == 'reddit']

In [28]:
train = pd.concat([twitter[:train_size//2],
                   reddit[:train_size//2]])
valid = pd.concat([twitter[train_size//2:train_size//2+test_size//2],
                   reddit[train_size//2:train_size//2+test_size//2]])
test = pd.concat([twitter[-(test_size//2):],
                   reddit[-(test_size//2):]])

# Save

In [4]:
df.to_csv(OUTPUT_DIR + 'df.csv')
train.to_csv(OUTPUT_DIR + 'train.csv')
valid.to_csv(OUTPUT_DIR + 'valid.csv')
test.to_csv(OUTPUT_DIR + 'test.csv')